In [1]:
!unzip "/content/drive/MyDrive/processed.zip" -d "/content/data"


Archive:  /content/drive/MyDrive/processed.zip
   creating: /content/data/processed/
   creating: /content/data/processed/density/
  inflating: /content/data/processed/density/IMG_1.npy  
  inflating: /content/data/processed/density/IMG_10.npy  
  inflating: /content/data/processed/density/IMG_100.npy  
  inflating: /content/data/processed/density/IMG_101.npy  
  inflating: /content/data/processed/density/IMG_102.npy  
  inflating: /content/data/processed/density/IMG_103.npy  
  inflating: /content/data/processed/density/IMG_104.npy  
  inflating: /content/data/processed/density/IMG_105.npy  
  inflating: /content/data/processed/density/IMG_106.npy  
  inflating: /content/data/processed/density/IMG_107.npy  
  inflating: /content/data/processed/density/IMG_108.npy  
  inflating: /content/data/processed/density/IMG_109.npy  
  inflating: /content/data/processed/density/IMG_11.npy  
  inflating: /content/data/processed/density/IMG_110.npy  
  inflating: /content/data/processed/density/IM

In [17]:
from torch.utils.data import Dataset
from PIL import Image

class CrowdDataset(Dataset):
    def __init__(self, img_paths, dens_paths, transform=None):
        self.img_paths = img_paths
        self.dens_paths = dens_paths
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert("RGB")
        dens = np.load(self.dens_paths[idx])  # (H,W)

        dens = torch.tensor(dens, dtype=torch.float32).unsqueeze(0)  # → shape: (1,H,W)

        if self.transform:
            img = self.transform(img)

        return img, dens


In [18]:
import glob
from torchvision import transforms

images_path = "/content/data/processed/images/*.jpg"
density_path = "/content/data/processed/density/*.npy"

img_files = sorted(glob.glob(images_path))
den_files = sorted(glob.glob(density_path))

transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = CrowdDataset(img_files, den_files, transform=transform)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import numpy as np

# ----------------------------------------------------
# 0. MOUNT GOOGLE DRIVE
# ----------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

# Folder where you want to save your training files
drive_folder = "/content/drive/MyDrive/CSRNet_Training/"
os.makedirs(drive_folder, exist_ok=True)

checkpoint_path = os.path.join(drive_folder, "csrnet_checkpoint.pth")
stats_path = os.path.join(drive_folder, "training_stats.npz")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# ----------------------------------------------------
# 1. MODEL (CSRNet)
# ----------------------------------------------------
class CSRNet(nn.Module):
    def __init__(self):
        super(CSRNet, self).__init__()

        self.frontend = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=1), nn.ReLU(inplace=True)
        )

        self.backend = nn.Sequential(
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, 3, dilation=2, padding=2), nn.ReLU(inplace=True),
        )

        self.output_layer = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x


# ----------------------------------------------------
# 2. LOAD MODEL + RESUME CHECKPOINT
# ----------------------------------------------------
model = CSRNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

start_epoch = 1
loss_history = []
gt_history = []
pred_history = []

# If checkpoint exists → resume
if os.path.exists(checkpoint_path):
    print("🔄 Resuming training from checkpoint in Drive...")
    ckpt = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(ckpt["model"])
    optimizer.load_state_dict(ckpt["optimizer"])
    start_epoch = ckpt["epoch"] + 1

    if os.path.exists(stats_path):
        data = np.load(stats_path, allow_pickle=True)
        loss_history = list(data["loss_history"])
        gt_history = list(data["gt_history"])
        pred_history = list(data["pred_history"])
else:
    print("🆕 Starting training from scratch!")


# ----------------------------------------------------
# 3. TRAIN LOOP
# ----------------------------------------------------
num_epochs = 50
train_loader = DataLoader(dataset, batch_size=1, shuffle=True)

for epoch in range(start_epoch, num_epochs + 1):
    model.train()
    epoch_loss = 0.0
    total_gt = 0.0
    total_pred = 0.0

    pbar = tqdm(train_loader)

    for img, dens in pbar:
        img = img.to(device)
        dens = dens.to(device)

        optimizer.zero_grad()
        pred = model(img)

        # Resize to match GT size
        if pred.shape != dens.shape:
            pred = F.interpolate(pred, size=dens.shape[2:], mode="bilinear", align_corners=False)

        loss = F.mse_loss(pred, dens)
        loss.backward()
        optimizer.step()

        batch_loss = loss.item()
        epoch_loss += batch_loss

        gt_count = dens.sum().item()
        pred_count = pred.sum().item()

        total_gt += gt_count
        total_pred += pred_count

        pbar.set_description(
            f"Epoch {epoch}/{num_epochs} | loss={batch_loss:.6f} | gt={gt_count:.1f} | pred={pred_count:.1f}"
        )

    # Save epoch stats
    loss_history.append(epoch_loss / len(train_loader))
    gt_history.append(total_gt)
    pred_history.append(total_pred)

    print(f"\n--- Epoch {epoch}/{num_epochs} | AvgLoss={loss_history[-1]:.6f} "
          f"| GT={total_gt:.2f} | Pred={total_pred:.2f} | Diff={abs(total_gt-total_pred):.2f} ---\n")

    # Save checkpoint to Google Drive
    torch.save({
        "epoch": epoch,
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict()
    }, checkpoint_path)

    # Save stats
    np.savez(stats_path,
             loss_history=np.array(loss_history, dtype=float),
             gt_history=np.array(gt_history, dtype=float),
             pred_history=np.array(pred_history, dtype=float))

    print("💾 Saved checkpoint + stats to Google Drive!\n")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🆕 Starting training from scratch!


Epoch 1/50 | loss=0.011227 | gt=768.7 | pred=451.9: 100%|██████████| 300/300 [01:29<00:00,  3.35it/s]



--- Epoch 1/50 | AvgLoss=0.026105 | GT=161423.18 | Pred=106734.42 | Diff=54688.77 ---

💾 Saved checkpoint + stats to Google Drive!



Epoch 2/50 | loss=0.035620 | gt=339.1 | pred=141.4: 100%|██████████| 300/300 [01:27<00:00,  3.42it/s]



--- Epoch 2/50 | AvgLoss=0.019908 | GT=161423.18 | Pred=172664.20 | Diff=11241.01 ---

💾 Saved checkpoint + stats to Google Drive!



...

Epoch 46/50 | loss=0.001170 | gt=184.6 | pred=160.2: 100%|██████████| 300/300 [01:27<00:00,  3.42it/s]



--- Epoch 46/50 | AvgLoss=0.007546 | GT=161423.18 | Pred=159053.03 | Diff=2370.15 ---

💾 Saved checkpoint + stats to Google Drive!



Epoch 47/50 | loss=0.013678 | gt=339.1 | pred=261.2: 100%|██████████| 300/300 [01:27<00:00,  3.42it/s]



--- Epoch 47/50 | AvgLoss=0.007868 | GT=161423.18 | Pred=161546.41 | Diff=123.23 ---

💾 Saved checkpoint + stats to Google Drive!



Epoch 48/50 | loss=0.000798 | gt=198.1 | pred=267.3:  88%|████████▊ | 265/300 [01:16<00:11,  3.11it/s]

In [14]:
print("Images:", len(img_files))
print("Density maps:", len(den_files))
print(img_files[:5], "\n", den_files[:5])


Images: 300
Density maps: 300
['/content/data/processed/images/IMG_1.jpg', '/content/data/processed/images/IMG_10.jpg', '/content/data/processed/images/IMG_100.jpg', '/content/data/processed/images/IMG_101.jpg', '/content/data/processed/images/IMG_102.jpg'] 
 ['/content/data/processed/density/IMG_1.npy', '/content/data/processed/density/IMG_10.npy', '/content/data/processed/density/IMG_100.npy', '/content/data/processed/density/IMG_101.npy', '/content/data/processed/density/IMG_102.npy']


In [21]:
import torch
import os
import torch.nn as nn

# Loss function for CSRNet
criterion = nn.MSELoss()

checkpoint_dir = "/content/drive/MyDrive/CSRNet_Training"
checkpoint_file = "csrnet_checkpoint.pth"   # <-- change this to your file name

# Load checkpoint
checkpoint_path = os.path.join(checkpoint_dir, checkpoint_file)
checkpoint = torch.load(checkpoint_path, map_location=device)

model.load_state_dict(checkpoint["model"])
optimizer.load_state_dict(checkpoint["optimizer"])
start_epoch = checkpoint.get("epoch", 0) + 1   # If epoch stored, continue, else start from 1

print(f"✅ Loaded checkpoint, resuming from epoch {start_epoch}")

# Continue training
num_epochs = 60

for epoch in range(start_epoch, num_epochs + 1):
    model.train()
    epoch_loss = 0.0
    gt_sum = 0.0
    pred_sum = 0.0

    pbar = tqdm(train_loader)
    for img, dens in pbar:
        img = img.to(device)
        dens = dens.to(device)

        optimizer.zero_grad()
        outputs = model(img)
        outputs = F.interpolate(outputs, size=dens.shape[2:], mode='bilinear', align_corners=False)

        loss = criterion(outputs, dens)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        gt_sum += dens.sum().item()
        pred_sum += outputs.sum().item()

        pbar.set_description(f"Epoch {epoch}/100 | loss={loss.item():.6f} | gt={dens.sum().item():.1f} | pred={outputs.sum().item():.1f}")

    print(f"\n--- Epoch {epoch}/100 | AvgLoss={epoch_loss/len(train_loader):.6f} | GT={gt_sum:.2f} | Pred={pred_sum:.2f} | Diff={abs(gt_sum-pred_sum):.2f} ---")

    save_path = os.path.join(checkpoint_dir, f"csrnet_epoch_{epoch}.pth")
    torch.save({
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "epoch": epoch
    }, save_path)

    print("💾 Saved checkpoint!")


✅ Loaded checkpoint, resuming from epoch 51


Epoch 51/100 | loss=0.004553 | gt=620.8 | pred=777.1:  23%|██▎       | 69/300 [00:20<01:09,  3.31it/s]


KeyboardInterrupt: 

In [23]:
from google.colab import files
files.download('/content/drive/MyDrive/CSRNet_Training/csrnet_checkpoint.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>